In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print("Pandas Ver.: " + pd.__version__)
print("Seaborn Ver.: " + sns.__version__)
print("Numpy Ver.: " + np.__version__)

Pandas Ver.: 1.5.3
Seaborn Ver.: 0.12.2
Numpy Ver.: 1.24.2


# Introduction

This is a dataset obtained from Kaggle, and it can be found here: [Steam Store Games (Clean Dataset)](https://www.kaggle.com/datasets/nikdavis/steam-store-games). The data was gathered around May 2019, utilizing the Steam Store and SteamSpy APIs, as stated by the dataset's author on the description.

This is an exploratory analysis of this dataset, mainly utilizing Pandas and its functions. I will try to answer the following questions:

- Top 20 games with the most positive ratings
- Top 20 games with the most negative ratings
- Top 20 games with the highest average playtime
- The most recurring tags/genres
- Publishers with the highest number of games
- Time series of releases (years with the most releases, periods where most games are released in a year etc.)
- What is the correlation between genre and playtime
- What is the correlation between genre and ratings
- What are the most owned games
- What are the least owned games
- Is there a relationship between prices, playtime and reviews, that perhaps points to a sunken cost fallacy?

In [2]:
steam_df = pd.read_csv("./data/steam.csv")
steam_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
steam_df.tail()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19
27074,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,4,0,0,0,0-20000,5.19


Getting a summary of the statistics in my dataset

In [4]:
steam_df.describe()

,appid,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,2.707500e+04,27075.000000,27075.000000,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075.000000
mean,5.962035e+05,0.981127,0.354903,45.248864,1.000559e+03,211.027147,149.804949,146.05603,6.078193
std,2.508942e+05,0.136081,2.406044,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,7.874922
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000
25%,4.012300e+05,1.000000,0.000000,0.000000,6.000000e+00,2.000000,0.000000,0.00000,1.690000
50%,5.990700e+05,1.000000,0.000000,7.000000,2.400000e+01,9.000000,0.000000,0.00000,3.990000
75%,7.987600e+05,1.000000,0.000000,23.000000,1.260000e+02,42.000000,0.000000,0.00000,7.190000
max,1.069460e+06,1.000000,18.000000,9821.000000,2.644404e+06,487076.000000,190625.000000,190625.00000,421.990000


In [5]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

Verifying how many lines and columns my dataset has

In [6]:
print(steam_df.shape)

(27075, 18)


Checking if there are any null values in my dataframe. This is good practice, although this dataset was already cleaned by the author.

In [7]:
print(steam_df.isnull().sum())

appid               0
name                0
release_date        0
english             0
developer           0
publisher           0
platforms           0
required_age        0
categories          0
genres              0
steamspy_tags       0
achievements        0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
owners              0
price               0
dtype: int64


Columns appid and steamspy_tags will be dropped. The former because it doesn't have any relevant information for this exploratory analysis, and the latter because it has very similar data to the genres column, and that is the chosen column to be used in this analysis.

In [8]:
steam_df = steam_df.drop(["appid", "steamspy_tags"], axis=1)
steam_df.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,10000000-20000000,7.19
1,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,5000000-10000000,3.99
2,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,5000000-10000000,3.99
3,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,5000000-10000000,3.99
4,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,5000000-10000000,3.99


Altering the genres column inplace. Since this column has many genres separated by semicolons, we will disregard the ones that come after the first and consider that the main genre.

In [9]:
split_genres = steam_df["genres"].str.split(";", n=1, expand=True)
steam_df["genres"] = split_genres[0]
steam_df.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,10000000-20000000,7.19
1,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,5000000-10000000,3.99
2,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,5000000-10000000,3.99
3,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,5000000-10000000,3.99
4,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,5000000-10000000,3.99


There's a lot of zeroes in the column required_age, so a query is run to check whether that is the situation for the entire column or not, in case it's bad data

In [10]:
steam_df.query("required_age > 0")

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
46,Quake IV,2011-08-04,1,RavenSoft / id Software,Bethesda Softworks,windows,18,Single-player;Multi-player;Steam Cloud,Action,0,891,171,36,32,200000-500000,12.99
132,Company of Heroes - Legacy Edition,2007-07-17,1,Relic Entertainment,SEGA,windows,18,Single-player;Multi-player;Steam Trading Cards...,Violent,0,4772,320,175,186,2000000-5000000,0.00
133,"Warhammer® 40,000: Dawn of War® - Game of the ...",2007-08-07,1,Relic Entertainment,SEGA,windows,16,Single-player;Multi-player;Steam Trading Cards...,Strategy,0,3485,206,309,340,1000000-2000000,9.99
134,"Warhammer® 40,000: Dawn of War® - Dark Crusade",2007-08-07,1,Relic Entertainment,SEGA,windows,16,Single-player;Multi-player;Steam Cloud,Strategy,0,2849,133,543,620,1000000-2000000,9.99
136,Condemned: Criminal Origins,2012-10-29,1,Monolith,SEGA,windows,18,Single-player,Action,0,1243,221,312,312,200000-500000,9.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26707,Faux,2019-03-18,1,Pelagames,Pelagames,windows,12,Single-player;Steam Achievements;Partial Contr...,Action,10,3,1,0,0,0-20000,2.09
26728,Roach Killer,2019-04-15,1,Highlight Visual Effects Development,Highlight Visual Effects Development,windows,16,Single-player;Steam Achievements;Steam Leaderb...,Action,1,1,2,0,0,0-20000,4.79
26732,oscar and paul,2019-03-15,1,NGS GAMES STUDIO;PIXEL ART GAME,NGS GAMES STUDIO,windows,12,Single-player,Action,0,0,1,0,0,0-20000,0.79
26983,Adecke - Cards Games Deluxe,2019-04-20,1,Philippe Pelchat,Philippe Pelchat,windows;mac;linux,12,Single-player,Casual,0,3,1,0,0,0-20000,7.19


Discovering the top 20 games with the most positive reviews on Steam. First, the 20 largest values in the positive_ratings column will be filtered.

In [11]:
steam_df['positive_ratings'].nlargest(n=20)

25       2644404
22        863507
19        515879
12836     496184
121       363721
2478      329061
1467      308657
3362      292574
1120      255600
21        251789
5235      251178
2031      242561
1025      237303
1634      226541
2016      220370
2964      202930
1596      176769
8129      169580
4712      145035
903       144595
Name: positive_ratings, dtype: int64

Then, a query is applied to select only those games whose quantity of positive ratings is above the last number retrieved in the filter. In this case, that would be 144595.

In [12]:
steam_df.query("positive_ratings >= 144595").sort_values(by="positive_ratings", ascending=False)

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
25,Counter-Strike: Global Offensive,2012-08-21,1,Valve;Hidden Path Entertainment,Valve,windows;mac;linux,0,Multi-player;Steam Achievements;Full controlle...,Action,167,2644404,402313,22494,6502,50000000-100000000,0.00
22,Dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action,0,863507,142079,23944,801,100000000-200000000,0.00
19,Team Fortress 2,2007-10-10,1,Valve,Valve,windows;mac;linux,0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action,520,515879,34036,8495,623,20000000-50000000,0.00
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,1,PUBG Corporation,PUBG Corporation,windows,0,Multi-player;Online Multi-Player;Stats,Action,37,496184,487076,22938,12434,50000000-100000000,26.99
121,Garry's Mod,2006-11-29,1,Facepunch Studios,Valve,windows;mac;linux,0,Single-player;Multi-player;Co-op;Cross-Platfor...,Indie,29,363721,16433,12422,1875,10000000-20000000,6.99
2478,Grand Theft Auto V,2015-04-13,1,Rockstar North,Rockstar Games,windows,18,Single-player;Multi-player;Steam Achievements;...,Action,77,329061,139308,9837,4834,10000000-20000000,24.99
1467,PAYDAY 2,2013-08-13,1,OVERKILL - a Starbreeze Studio.,Starbreeze Publishing AB,windows;linux,18,Single-player;Multi-player;Co-op;Online Co-op;...,Action,1130,308657,56523,3975,890,10000000-20000000,7.49
3362,Unturned,2017-07-07,1,Smartly Dressed Games,Smartly Dressed Games,windows;mac;linux,0,Single-player;Online Multi-Player;Online Co-op...,Action,46,292574,31482,3248,413,20000000-50000000,0.00
1120,Terraria,2011-05-16,1,Re-Logic,Re-Logic,windows;mac;linux,0,Single-player;Multi-player;Online Multi-Player...,Action,88,255600,7797,5585,1840,5000000-10000000,6.99
21,Left 4 Dead 2,2009-11-19,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,70,251789,8418,1615,566,10000000-20000000,7.19


Unsurprisingly, the podium is occupied by the wildly popular PC games Counter-Strike: Global Offensive, Dota 2 and Team Fortress 2, all three developed by Steam's own parent company, Valve Software.

Now, going to the opposite end, discovering the top 20 games with the most negative reviews on Steam. Filtering for the the 20 largest values in the negative_ratings column.

In [13]:
steam_df["negative_ratings"].nlargest(20)

12836    487076
25       402313
22       142079
2478     139308
7809      91664
1506      77169
4712      66603
5235      59620
1467      56523
2563      56488
2016      53709
5916      45377
6046      44985
12965     35294
1607      34416
19        34036
8129      32407
3362      31482
3251      30987
3347      25920
Name: negative_ratings, dtype: int64

Applying a query to select only those games whose quantity of negative ratings is above the last number retrieved in the filter, 25920.

In [14]:
steam_df.query("negative_ratings >= 25920").sort_values(by="negative_ratings", ascending=False)

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,1,PUBG Corporation,PUBG Corporation,windows,0,Multi-player;Online Multi-Player;Stats,Action,37,496184,487076,22938,12434,50000000-100000000,26.99
25,Counter-Strike: Global Offensive,2012-08-21,1,Valve;Hidden Path Entertainment,Valve,windows;mac;linux,0,Multi-player;Steam Achievements;Full controlle...,Action,167,2644404,402313,22494,6502,50000000-100000000,0.00
22,Dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action,0,863507,142079,23944,801,100000000-200000000,0.00
2478,Grand Theft Auto V,2015-04-13,1,Rockstar North,Rockstar Games,windows,18,Single-player;Multi-player;Steam Achievements;...,Action,77,329061,139308,9837,4834,10000000-20000000,24.99
7809,Z1 Battle Royale,2018-02-28,1,NantG,Daybreak Game Company,windows,0,Multi-player;Online Multi-Player;In-App Purchases,Action,0,111434,91664,3142,162,10000000-20000000,0.00
1506,DayZ,2018-12-13,1,Bohemia Interactive,Bohemia Interactive,windows,18,Multi-player;Online Multi-Player;Steam Worksho...,Action,0,118435,77169,7235,2798,2000000-5000000,33.99
4712,ARK: Survival Evolved,2017-08-27,1,Studio Wildcard;Instinct Games;Efecto Studios;...,Studio Wildcard,windows;mac;linux,0,Single-player;Multi-player;MMO;Co-op;Steam Ach...,Action,32,145035,66603,14154,1470,5000000-10000000,44.99
5235,Tom Clancy's Rainbow Six® Siege,2015-12-01,1,Ubisoft Montreal,Ubisoft,windows,0,Single-player;Multi-player;Co-op;Steam Trading...,Action,0,251178,59620,7504,2543,5000000-10000000,16.99
1467,PAYDAY 2,2013-08-13,1,OVERKILL - a Starbreeze Studio.,Starbreeze Publishing AB,windows;linux,18,Single-player;Multi-player;Co-op;Online Co-op;...,Action,1130,308657,56523,3975,890,10000000-20000000,7.49
2563,No Man's Sky,2016-08-12,1,Hello Games,Hello Games,windows,0,Single-player;Multi-player;Online Multi-Player...,Action,27,63444,56488,2004,1222,1000000-2000000,39.99


Popular video games, especially those with live-service and multiplayer components, can often be very divisive or controversial due to the fickle nature of the player bases and the state of their satisfaction or dissatisfaction with the games as they get updated and changed. Thus Counter-Strike: Global Offensive and Dota 2 occupy 2 of the 3 spots of games with the most negative reviews.